# Getting available actions

**Note** this notebook requres that there are pending actions in the database. So go create some first. 

Actions are configured in `app\configurations` directory. When the UI makes a request to get the actions available for an object, the `app/ajaxviews/actions.py` loads the yaml, and interprets what actions that the selected object is available to take. Those actions are sent to the user. 

### Two times when actions come up:
but only one discussed here. 
* ~~web interaction that creates jobs~~
* **azure functions that resolve actions**

In [1]:
import sys
import numpy as np
import pandas as pd


sys.path.append("../..")
sys.path.append("..")
import yaml, ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()


from app.connectors import cmdb_graph
from app.objects import time as t


c = cmdb_graph.CosmosdbClient()


executing local windows deployment


In [2]:
time = t.Time(c)
time.get_current_UTU()
time

< time at: 2023-07-14T17:52:42.264448+00:00 UTU:28298 >

In [3]:
time.get_global_actions()
actions_df = pd.DataFrame(time.actions)
actions_df

""


All actions require both an agent, job and action. The properties of the action determine how it affects other things. 


In [4]:
for i in actions_df.index:
    action = t.Action(c,actions_df.loc[i])
    print(action)
    

In [5]:
action.action.get('augments_self_properties')

NameError: name 'action' is not defined

In [ ]:
pd.DataFrame([action.agent, action.resolve_augments_self_properties()], index = ['og','updated'])

,health,wealth,isIdle,name,objid,conformity,literacy,aggression,constitution,isIn,industry,factionLoyalty,username,objtype
og,0.70,0.4882,false,Shiyaos Hou,3209702584262,0.631,0.612,0.399,0.33,8747874146271,0.3645,0.664,Billmanserver,pop
updated,0.75,0.4382,false,Shiyaos Hou,3209702584262,0.631,0.612,0.399,0.33,8747874146271,0.3645,0.664,Billmanserver,pop


In [ ]:
action.add_updates_to_c(time)
action.c.stack

["g.V().has('objid','3209702584262').property('health',0.7).property('wealth',0.4882)",
 "g.V().has('objid','3209702584262').property('isIdle','true')",
 "g.V().has('objid','3209702584262').outE('takingAction').has('actionType','healthcare_initiatives').has('weight','28299').property('status','resolved')"]

In [ ]:
action.data

{'nodes': [{'objid': '0675471245432',
   'name': 'job',
   'label': 'event',
   'text': 'The pop (Shiyaos Hou) has completed healthcare_initiatives',
   'visibleTo': 'Billmanserver',
   'time': 28298,
   'username': 'event'}],
 'edges': ["g.V().has('objid','0675471245432').addE('completed').to(g.V().has('objid','3209702584262'))"]}

Veryify that actions are resolved or not

In [6]:
action.resolve_action()

NameError: name 'action' is not defined

In [12]:
action.agent['username']

'Billmanserver'